In [267]:
import pandas as pd
out=pd.read_csv('./data/views.dat',sep='::',header = None,names=['userid','itemid','time'])
# out

/Users/wangrui/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
# out.pivot(index='userid',columns='itemid',values='1')

In [266]:
# userCF
item_users = dict()
for row in out.itertuples():
#     print(row[0],row[1],row[2],row[3])
    item_users.setdefault(row[2], set())
    item_users[row[2]].add((row[1],row[3]))
# print("ok")
# item_users

In [241]:
# len(item_users.items())
useritemcount=dict()        # 每个用户的访问总量
useritemcount = out.groupby(out['userid']).size().to_dict()
# useritemcount

# itemusercount=dict()        # 每个item的访问用户总量
# itemusercount = out.groupby(out['itemid']).size().to_dict()
# itemusercount

In [264]:
# ################################################################################
# userCF 算法
import datetime
uanduitem=dict()
a = out[['itemid','userid']].sort_values(by=['itemid'])
uids = pd.Series(a['itemid']).unique()
# b = a[a['itemid']==uid]
uidbyitemid = pd.Series(b['userid'])
start = datetime.datetime.now()
print('start==',start)
for uid in uids:
    b = a[a['itemid']==uid]
    uidbyitemid = pd.Series(b['userid']).unique()
    for u in uidbyitemid:
        for v in uidbyitemid:
            if u ==v:
                continue
            uanduitem.setdefault(u, {})
            uanduitem[u].setdefault(v, 0)
            uanduitem[u][v] += 1
end = datetime.datetime.now()
print('end==',end)
print('times==',end-start)

In [262]:
import math
W = dict()
start = datetime.datetime.now()
print(start)
for u,ru in uanduitem.items():
    for v, cuv in ru.items():
        W.setdefault(u, {})
        W[u].setdefault(v, cuv / math.sqrt(useritemcount[u] * useritemcount[v]))
end = datetime.datetime.now()
print(end)
print(end-start)
# ################################################################################

2018-03-23 10:10:44.025293
2018-03-23 10:10:45.817210
0:00:01.791917


In [ ]:
# ################################################################################
# userCF-IIF 算法
import datetime
uanduitem=dict()
a = out[['itemid','userid']].sort_values(by=['itemid'])
uids = pd.Series(a['itemid']).unique()
# b = a[a['itemid']==uid]
uidbyitemid = pd.Series(b['userid'])
start = datetime.datetime.now()
print('start==',start)
for uid in uids:
    b = a[a['itemid']==uid]
    uidbyitemid = pd.Series(b['userid']).unique()
    usercount = len(uidbyitemid)
    for u in uidbyitemid:
        for v in uidbyitemid:
            if u ==v:
                continue
            uanduitem.setdefault(u, {})
            uanduitem[u].setdefault(v, 0)
            uanduitem[u][v] += 1 / math.log(1 + usercount)    #userCF 和userCF-IIF唯一的区别
end = datetime.datetime.now()
print('end==',end)
print('times==',end-start)

In [ ]:
import math
W = dict()
start = datetime.datetime.now()
print(start)
for u,ru in uanduitem.items():
    for v, cuv in ru.items():
        W.setdefault(u, {})
        W[u].setdefault(v, cuv / math.sqrt(useritemcount[u] * useritemcount[v]))
end = datetime.datetime.now()
print(end)
print(end-start)
# ################################################################################

In [259]:
useritemslist = dict()
for i in item_users.keys():
    a = out[['userid','itemid']]
    b = pd.Series(a[a['userid']==i]['itemid']).unique()
    useritemslist[i] = b

In [330]:
#可同时用于userCF 和userCF-IIF
def Recommend(user, K=20, N=20):
    from operator import itemgetter, attrgetter
    rank = dict()
    rvi =1
    interacted_items = useritemslist[user]
    if user in W.keys():
        for v, wuv in sorted(W[user].items(), key=itemgetter(1),reverse=True)[0:K]:
            for i in useritemslist[v]:
                 if i in interacted_items:
                     #we should filter items user interacted before
                     continue
                 rank.setdefault(i, 0)
                 rank[i] += wuv * rvi
                 
        ret = sorted(rank.items(),key=itemgetter(1),reverse=True)
        # ret = sorted([(k,v)for k,v in rank.items()],key=itemgetter(1),reverse=True)
        return ret[:N]
    else:
        return []
# Recommend(1)
# Recommend(10)

In [ ]:
# ##################################################################################
# userCF-TIME 算法, 加入了时间衰减因素
import datetime
uanduitem=dict()
alpha = 0.5
a = out[['itemid','userid']].sort_values(by=['itemid'])
uids = pd.Series(a['itemid']).unique()
# b = a[a['itemid']==uid][['userid','time']]
# uidbyitemid = pd.Series(b['userid']).unique()
start = datetime.datetime.now()
print('start==',start)
for uid in uids:
    uidtimebyitemid = a[a['itemid']==uid][['userid','time']]
    uidtimebyitemid = pd.Series(b['userid']).unique()
    usercount = uidtimebyitemid.shape[0]
    for u in uidtimebyitemid:
        for v in uidtimebyitemid:
            if u[1] == v[1:
                continue
            uanduitem.setdefault(u, {})
            uanduitem[u].setdefault(v, 0)
            uanduitem[u][v] += 1 / (1 + alpha *(u[2]-v[2]))    #userCF 和userCF-IIF唯一的区别
end = datetime.datetime.now()
print('end==',end)
print('times==',end-start)

In [ ]:
import math
W = dict()
start = datetime.datetime.now()
print(start)
for u,ru in uanduitem.items():
    for v, cuv in ru.items():
        W.setdefault(u, {})
        W[u].setdefault(v, cuv / math.sqrt(useritemcount[u] * useritemcount[v]))
end = datetime.datetime.now()
print(end)
print(end-start)
# ################################################################################

In [328]:
#可同时用于userCF 和userCF-IIF
useritemstimeslist = dict()
for i in item_users.keys():
    a = out[['userid','itemid','time']]
    b = a[a['userid']==i][['itemid','time']]
    useritemstimeslist[i] = b
def RecommendTime(user, alpha, t0, K=20, N=20): #t0时间
    from operator import itemgetter, attrgetter
    rank = dict()
    rvi =1
    interacted_items = useritemstimeslist[user]['itemid'].unique()
    if user in W.keys():
        for v, wuv in sorted(W[user].items(), key=itemgetter(1),reverse=True)[0:K]:
            for i in useritemstimeslist[v]:
                 if i[1] in interacted_items:
                     #we should filter items user interacted before
                     continue
                 rank.setdefault(i, 0)
                 rank[i] += wuv /(1+alpha*(t0-i[2]))
                 
        ret = sorted(rank.items(),key=itemgetter(1),reverse=True)
        return ret[:N]
    else:
        return []
import time

# Recommend(1,0.5,int(time.time()))
# Recommend(10)

In [329]:
useritemstimeslist = dict()
for i in item_users.keys():
    a = out[['userid','itemid','time']]
    b = a[a['userid']==i][['itemid','time']]
    useritemstimeslist[i] = b
#     print(b)

In [319]:
type(useritemstimeslist[10])
useritemstimeslist[10]['itemid'].unique()

array([117,  47, 168,  83,  86, 141,  32,  26, 145, 111,   1,  53,  50,
       156,  56, 144,  99,  74, 122, 116,  70, 118,  20,  60,  73, 119,
        23,   2, 128,  39, 134,  10, 132, 121,  92, 130, 133, 120,   6,
        28,  45,   8, 129,  65, 148, 167,  62,  58, 164, 154, 153,  59,
       172, 127,  80, 163,  43,   9, 160,  61,  72,  90,   7, 136, 126,
       110, 107, 162,  87,   5, 155, 140, 166,  22,  81,  48, 102,  35,
        31,  11,  44,  63,  64, 112,  95,  52,  12, 146,  15, 115, 170,
        76, 114, 159,  29,  54,  38,  68,  34,  78, 142, 124, 150, 147,
        88, 101,  85,  51, 174, 173,  19, 125, 131, 138,  21,  24, 149,
       108,  36,  71,  25,  42,  96, 113, 151, 161,  46,  89,  97, 157,
        14,  82])